In [1]:
open System.Linq

type MerkleHash = byte[]

type MerkleNode = { 
    Hash: MerkleHash;
    Left: MerkleNode option;
    Right: MerkleNode option;
    Id: string;
} with 
    member this.IsLeaf = (Option.isNone this.Left) && (Option.isNone this.Right)
//true if the option is None.

type MerkleTree = {
    RootNode: MerkleNode;
    Depth: int;
    Leaves: int;
} with 
    member this.RootHash = this.RootNode.Hash 

module Constants =
    let hashLength = 32

module MerkleHash =
    let ofByteArray (buffer:byte[]) : MerkleHash = System.Security.Cryptography.SHA256.Create().ComputeHash(buffer)

    let ofString (buffer:string) = System.Text.Encoding.UTF8.GetBytes(buffer) |> ofByteArray

    let concat (hash1:MerkleHash) (hash2:MerkleHash) = Array.concat [hash1; hash2] |> ofByteArray

    let toString (mh:MerkleHash) = System.BitConverter.ToString(mh).Replace("-", "")

    let eq (hash1:MerkleHash) (hash2:MerkleHash) = hash1 = hash2 || hash1.SequenceEqual(hash2)


module MerkleNode =
    let leaf hash id = { Hash = hash; Id = id; Left = None; Right = None; }

    let ofNodes left right =
        let mh, leftId, rightId =
            match left, right with
            | l, Some r -> MerkleHash.concat l.Hash r.Hash, l.Id, r.Id
            | l, None -> l.Hash, l.Id, "<None>"
        let id = sprintf "\"%s\"-\"%s\"" leftId rightId
        { Hash = mh; Id = id; Left = Some left; Right = right; }

    let ofNodesTuple (left, right) = ofNodes left right

    let rec createBranchWithDepth depth leaf =
        match depth with
        | 0 -> failwith "Cannot create branch with depth 0"
        | 1 -> leaf
        | _ -> ofNodes (createBranchWithDepth (depth - 1) leaf) None

    let verifyHash node =
            match node.Left, node.Right with
            | None, None -> true
            | None, _ -> failwith "Left branch must be a node  if right branch is a node"
            | Some left, None -> MerkleHash.eq node.Hash left.Hash
            | Some left, Some right -> 
                MerkleHash.concat left.Hash right.Hash 
                |> MerkleHash.eq node.Hash

    let eq node1 node2 = MerkleHash.eq node1.Hash node2.Hash


module MerkleTree = 
    module private Append =
        type AppendAction = Handled of MerkleNode | Unhandled of MerkleNode
        type NodeType = 
            | Leaf 
            | FullInternalEdge 
            | EmptyInternalEdge of left: MerkleNode 
            | EmptyInternal of left: MerkleNode 
            | FullInternal of left:MerkleNode * right: MerkleNode

        let getNodeType node = 
            match node.Left, node.Right with
            | None, None -> Leaf
            | None, _ -> failwith "Invalid tree with only left node"
            | Some l, None -> 
                match l.IsLeaf with
                | true -> EmptyInternalEdge l
                | false -> EmptyInternal l
            | Some l, Some r ->
                match l.IsLeaf with
                | true -> FullInternalEdge
                | false -> FullInternal (l, r)

        
        let append leaf tree =
            let rec tryAppend currentHeight node =
                let nextHeight = currentHeight - 1
                let nodeType = getNodeType node
                match nodeType with
                | Leaf -> Unhandled node
                | EmptyInternalEdge l -> Handled (MerkleNode.ofNodes l (Some leaf))
                | FullInternalEdge -> Unhandled node
                | EmptyInternal l -> 
                    let action = tryAppend nextHeight l
                    match action with
                    | Handled n -> Handled (MerkleNode.ofNodes n None)
                    | Unhandled _ -> 
                        //create new path to the right that is the full height
                        let r = MerkleNode.createBranchWithDepth nextHeight leaf
                        Handled (MerkleNode.ofNodes l (Some r))
                | FullInternal (l, r) -> 
                    let action = tryAppend nextHeight r
                    //if right path changed, create new node. otherwise use old node
                    match action with
                    | Handled n -> Handled (MerkleNode.ofNodes l (Some n))
                    | Unhandled _ -> Unhandled node

            let appendResult = tryAppend tree.Depth tree.RootNode

            match appendResult with
            | Handled n -> { RootNode = n; Depth = tree.Depth; Leaves = tree.Leaves + 1; }
            | Unhandled n -> 
                let right = MerkleNode.createBranchWithDepth tree.Depth leaf
                let newRoot = MerkleNode.ofNodes n (Some right)
                { RootNode = newRoot; Depth = tree.Depth + 1; Leaves = tree.Leaves + 1; }


    let create leaves = 
        let rec loop depth nodes =
            match Seq.length nodes with
            | 1 -> { RootNode =  Seq.head nodes; Depth = depth; Leaves = (Seq.length leaves); }
            | _ -> 
                nodes
                |> Seq.chunkBySize 2
                |> Seq.map (fun chunk -> (Array.item 0 chunk, Array.tryItem 1 chunk))
                |> Seq.map MerkleNode.ofNodesTuple
                |> loop (depth + 1)
        loop 1 leaves

    let append = Append.append

    //returns the root hash of the recalculated path using hte given hash for the nth item
    let verifyNth n hash tree =
        let rec loop n height node =
            match height with
            | 0 -> failwith "Could not locate nth node due to empty tree"
            | 1 -> hash
            | _ -> 
                //todo this needs to use the NodeType to traverse the tree.
                //pull out generic traversal and pass in a fn for handling the node types
                let nextHeight = height - 1
                let totalDescendants = 2.0 ** ((float height) - 1.0)
                let midpoint = int (totalDescendants / 2.0)
                let nextN, nextNode = 
                    match n <= midpoint, node.Left, node.Right with
                    | true, Some l, _ -> n, l
                    | false, _, Some r -> (n - midpoint), r
                    | _ -> failwith "Attempting to find nth item which does not exist"
                loop nextN nextHeight nextNode
        loop n tree.Depth tree.RootNode
        
      

"MerkleHash is a **SHA256** hash"

In [2]:
let hash = MerkleHash.ofString "Test 1"
let hashString = MerkleHash.toString hash

printfn "%A" hashString

"26EA0AE294881F1260ECAFEC008426894E80BC4D7DC1CD6557AB9169E1A803EE"


"MerkleHash.ofString with different strings are not equal"

In [3]:
let h1 = MerkleHash.ofString "Test 1"
let h2 = MerkleHash.ofString "Test 2"
let h3 = MerkleHash.ofString "Test 3"

printfn "%A" (h1=h2)

false


In [5]:
let createNode text = MerkleHash.ofString text |> MerkleNode.leaf
let node1 = createNode "Test 1" "1"
let node2 = createNode "Test 2" "2"

"MerkleNode.ofNodes uses the concat of the hashes of its children" 

In [6]:
let leaf1 = node1
let leaf2 = node2

let node = MerkleNode.ofNodes leaf1 (Some leaf2)

let leaf1Hash = MerkleHash.toString leaf1.Hash
let leaf2Hash = MerkleHash.toString leaf2.Hash
let hashString' = MerkleHash.toString node.Hash

let h1 =  leaf1Hash = hashString'
let h2 =  leaf2Hash = hashString'

printfn "%A" h1
printfn "%A" h2

let h12 =  MerkleHash.concat leaf1.Hash leaf2.Hash, leaf1.Id, leaf2.Id 
printfn "%A" node.Hash
printfn "%A"  h12 



false
false
[|140uy; 255uy; 218uy; 40uy; 19uy; 253uy; 132uy; 168uy; 136uy; 5uy; 64uy; 41uy;
  147uy; 192uy; 202uy; 195uy; 70uy; 99uy; 227uy; 51uy; 233uy; 33uy; 227uy; 64uy;
  74uy; 134uy; 220uy; 202uy; 109uy; 76uy; 38uy; 207uy|]
([|140uy; 255uy; 218uy; 40uy; 19uy; 253uy; 132uy; 168uy; 136uy; 5uy; 64uy; 41uy;
   147uy; 192uy; 202uy; 195uy; 70uy; 99uy; 227uy; 51uy; 233uy; 33uy; 227uy; 64uy;
   74uy; 134uy; 220uy; 202uy; 109uy; 76uy; 38uy; 207uy|], "1", "2")


The Merkle root is included in the **block header**. With this scheme, it is possible to securely verify that a transaction has been accepted by the network (and get the number of confirmations) by downloading just the tiny block headers and Merkle tree -- downloading the entire block chain is unnecessary.